In [2]:
import psycopg2
import xml.etree.ElementTree as ET
from bs4 import BeautifulSoup
import requests
import sys
sys.path.append('../')
from src.utils import connect_db, get_table_name, print_tree

In [3]:
xml_dir = 'baseplate_320-BA-PF0-VR-00004_cond_upload.xml'

In [4]:
def parse_xml_to_dict(element):
    """
    Recursively parse a BeautifulSoup tag and return a dictionary representation.
    """
    # If the element has no sub-elements, return its text content
    if not element.find_all(recursive=False):
        return element.text.strip() if element.text else None

    # If the element has attributes, include them in the result
    result = {attr: value for attr, value in element.attrs.items()} if element.attrs else {}

    # Parse each sub-element recursively
    for sub_element in element.find_all(recursive=False):
        result[sub_element.name] = parse_xml_to_dict(sub_element)

    return result

## Attempt - Update XML file with the value from PostgreSQL
## test
We want to 
1. select
    - feature_name = Corner1
    - bp_name = FNAL3-2
    - value = X
2. retreive 
    - inspector = simurthy
3. replace "initiated_by_user" to inspector value

In [8]:
# Load your XML file
# Replace 'path/to/your/file.xml' with the actual file path
with open(xml_dir, 'r') as file:
    xml_content = file.read()

# Parse the XML content
xml_data = BeautifulSoup(xml_content, 'lxml')

# Use the function to parse the XML
run_tag = xml_data.find('run')
run_parsed_dict = parse_xml_to_dict(run_tag) if run_tag else {}

In [9]:
run_parsed_dict

{'run_name': 'Flatness/thickness measurements',
 'run_begin_timestamp': '2023-10-04 13:27:52.927779',
 'run_end_timestamp': '2023-10-04 13:27:52.927779',
 'initiated_by_user': 'None',
 'location': 'CMU',
 'comment_description': 'Measured flatness/thickness data'}

In [13]:
# %load ../view/view_a_cell.py
import psycopg2
import os
import sys
import csv
import sys
sys.path.append('../')
from src.utils import connect_db

conn = connect_db()
cursor = conn.cursor()

retrieving_col = input('The column name to retrieve: ')
condition_1 = input('conditions for feature_name: ')
condition_2 = input('conditions for value: ')
condition_3 = input('conditions for bp_name: ')

query = f"""
SELECT {retrieving_col} FROM bp_inspect
WHERE feature_name = %s AND value = %s AND bp_name = %s;
"""

search_conditions = (condition_1, condition_2, condition_3)

cursor.execute(query, search_conditions)

result = cursor.fetchall()
print(f'The {retrieving_col} met with your specificied conditions is ----- {result[0]}')




Set superuser password: /nhgcal
The column name to retrieve: inspector
conditions for feature_name: Corner1
conditions for value: X
conditions for bp_name: FNAL3-2
The inspector met with your specificied conditions is ----- ('simurthy',)


In [15]:
## tentative dictionary
### {database column name: xml file tag}

db_to_xml = {"bp_name":"name",
             "inspector": "initiated_by_user",
            "comment":"comment_description"}

In [16]:
db_column = 'inspector'
xml_tag = db_to_xml[db_column]
db_val = result[0][0]
db_val

'simurthy'

In [23]:
run_parsed_dict[xml_tag] = db_val
run_parsed_dict

{'run_name': 'Flatness/thickness measurements',
 'run_begin_timestamp': '2023-10-04 13:27:52.927779',
 'run_end_timestamp': '2023-10-04 13:27:52.927779',
 'initiated_by_user': 'simurthy',
 'location': 'CMU',
 'comment_description': 'Measured flatness/thickness data'}

In [24]:
## replace a value in xml-originated dictionary
for tag, text in run_parsed_dict.items():
    run_tag.find(tag).string = text

In [25]:
run_tag

<run>
<run_name>Flatness/thickness measurements</run_name>
<run_begin_timestamp>2023-10-04 13:27:52.927779</run_begin_timestamp>
<run_end_timestamp>2023-10-04 13:27:52.927779</run_end_timestamp>
<initiated_by_user>simurthy</initiated_by_user>
<location>CMU</location>
<comment_description>Measured flatness/thickness data</comment_description>
</run>

In [29]:
updated_xml = str(xml_data)
updated_xml

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><html><body><root xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">\n<header>\n<type>\n<extension_table_name>SI_MODULE_BASEPLATE</extension_table_name>\n<name>Si Module Baseplate Test Data</name>\n</type>\n<run>\n<run_name>Flatness/thickness measurements</run_name>\n<run_begin_timestamp>2023-10-04 13:27:52.927779</run_begin_timestamp>\n<run_end_timestamp>2023-10-04 13:27:52.927779</run_end_timestamp>\n<initiated_by_user>simurthy</initiated_by_user>\n<location>CMU</location>\n<comment_description>Measured flatness/thickness data</comment_description>\n</run>\n</header>\n<data_set>\n<comment_description>Upload measured data</comment_description>\n<version>1</version>\n<part mode="auto">\n<serial_number>320-BA-PF0-VR-00004</serial_number>\n<kind_of_part>PCB/Kapton Baseplate LD Full</kind_of_part>\n</part>\n<data>\n<flatness>0.062</flatness>\n<thickness>1.219</thickness>\n<grade>Green</grade>\n<comments>None</comments>\n</data>\n

In [35]:
## export to a new xml file and save it to a directory
file_name = 'v2_' + xml_dir
file_path = '../export/' + file_name

with open(file_path,"w") as file:
    file.write(updated_xml)

## NEXT

1. Confirm the XML tags -- Does the arrangement of tags remain the same?
2. Create a dictionary that lets us convert the column name and xml tags